In [1]:
import os
import os.path
import sys
from pathlib import Path
sys.path.insert(0, '..')
from elitetools import eddb
eddb.load_feeds()

# Found "C:\Users\komin\AppData\Local\Temp\elite-tools\archive-v6-commodities.json".
# Feeds.COMMODITIES records loaded:  373
# Found "C:\Users\komin\AppData\Local\Temp\elite-tools\archive-v6-listings.csv".
# Feeds.PRICES records loaded:  4191028
# Found "C:\Users\komin\AppData\Local\Temp\elite-tools\archive-v6-factions.jsonl".
# Feeds.FACTIONS records loaded:  77156
# Found "C:\Users\komin\AppData\Local\Temp\elite-tools\archive-v6-systems_populated.jsonl".
# Feeds.POPULATED_SYSTEMS records loaded:  20568
# Found "C:\Users\komin\AppData\Local\Temp\elite-tools\archive-v6-stations.jsonl".
# Feeds.STATIONS records loaded:  76104
# Found "C:\Users\komin\AppData\Local\Temp\elite-tools\scraped_listings.jsonl".
# Real-time listing records loaded:  325


In [77]:
origin_name="Exioce"
commodity_names=['Tea']
minimum_supply = 100

In [78]:
origin = eddb.find_system_by_name(origin_name)
commodities = eddb.commodity_details[eddb.commodity_details['name'].isin(commodity_names)][['id', 'name']] \
    .rename(columns={'name': 'commodity', 'id': 'commodity_id'})
commodities

,commodity_id,commodity
20,21,Tea


In [79]:
listings = eddb.commodity_listings_eod.merge(commodities, on='commodity_id') \
    [['station_id', 'commodity', 'supply', 'buy_price']] \
    .query(f'supply > {minimum_supply}') \
    .drop_duplicates()
listings

,station_id,commodity,supply,buy_price
2,4,Tea,73666,1446
3,5,Tea,90223,1168
5,7,Tea,9385750,1218
34,37,Tea,1822554,1196
35,38,Tea,385,1685
...,...,...,...,...
55178,79080,Tea,200,116
55193,79970,Tea,340,85
55194,80151,Tea,696,76
55197,80285,Tea,290,1596


In [80]:
pos_listings = listings.merge(eddb.station_details[['id', 'name', 'system_id', 'max_landing_pad_size', 'distance_to_star']] \
    .rename(columns={'id': 'station_id', 'name': 'station', 'max_landing_pad_size': 'Pad Size', 'distance_to_star': 'Sublight'}), on='station_id') \
    .merge(eddb.populated_systems[['id', 'name', 'x', 'y', 'z']] \
    .rename(columns={'id': 'system_id', 'name': 'system'}), on='system_id') \
    .drop(columns=['station_id', 'system_id'])

pos_listings

,commodity,supply,buy_price,station,Pad Size,Sublight,system,x,y,z
0,Tea,73666,1446,Trevithick Terminal,L,209.0,Wadjali,-34.81250,-42.84375,70.87500
1,Tea,90223,1168,Reilly Hub,L,171.0,Abukunin,-69.31250,-7.43750,61.93750
2,Tea,9385750,1218,Ackerman Market,L,273.0,Eravate,-42.43750,-3.15625,59.65625
3,Tea,1822554,1196,Hornby Terminal,L,221.0,Witchhaul,-13.25000,-52.06250,-65.87500
4,Tea,385,1685,Futen Spaceport,L,553.0,Fujin,-6.03125,-30.37500,-59.03125
...,...,...,...,...,...,...,...,...,...,...
1969,Tea,150,2505,J4T-2QH,L,1473.0,LFT 1300,32.65625,-16.03125,36.84375
1970,Tea,281,1255,L4F-1DZ,L,96973.0,Pleiades Sector IH-V c2-16,-92.00000,-143.50000,-341.31250
1971,Tea,500,3395,X5W-68N,L,32.0,Guarakma,98.59375,-32.12500,-102.12500
1972,Tea,340,85,Q5H-LVZ,L,913.0,Ciguru,-82.40625,3.87500,-8.90625


In [81]:
dist_listings = pos_listings.assign(distance = lambda x: eddb.distance(origin, x))
dist_listings

,commodity,supply,buy_price,station,Pad Size,Sublight,system,x,y,z,distance
0,Tea,73666,1446,Trevithick Terminal,L,209.0,Wadjali,-34.81250,-42.84375,70.87500,138.1
1,Tea,90223,1168,Reilly Hub,L,171.0,Abukunin,-69.31250,-7.43750,61.93750,180.3
2,Tea,9385750,1218,Ackerman Market,L,273.0,Eravate,-42.43750,-3.15625,59.65625,160.9
3,Tea,1822554,1196,Hornby Terminal,L,221.0,Witchhaul,-13.25000,-52.06250,-65.87500,132.6
4,Tea,385,1685,Futen Spaceport,L,553.0,Fujin,-6.03125,-30.37500,-59.03125,133.3
...,...,...,...,...,...,...,...,...,...,...,...
1969,Tea,150,2505,J4T-2QH,L,1473.0,LFT 1300,32.65625,-16.03125,36.84375,98.0
1970,Tea,281,1255,L4F-1DZ,L,96973.0,Pleiades Sector IH-V c2-16,-92.00000,-143.50000,-341.31250,399.0
1971,Tea,500,3395,X5W-68N,L,32.0,Guarakma,98.59375,-32.12500,-102.12500,138.5
1972,Tea,340,85,Q5H-LVZ,L,913.0,Ciguru,-82.40625,3.87500,-8.90625,193.4


In [82]:
top_count = 5
listings_ranked = dist_listings.assign(rnk = dist_listings.groupby('commodity')['distance'] \
        .rank(method='first', ascending=True)) \
        .query(f'rnk <= {top_count}') \
        .sort_values(by=['distance', 'commodity', 'station'], ascending=[True, True, True]) \
        .drop_duplicates() \
        .reset_index() \
        .drop(columns=['x', 'y', 'z', 'index', 'rnk'])
listings_ranked

,commodity,supply,buy_price,station,Pad Size,Sublight,system,distance
0,Tea,3009,1316,Rafferty's Mobius,L,399.0,Azrael,8.4
1,Tea,914,1232,Seares Port,L,9446.0,Mu Horologii,15.3
2,Tea,4447,1316,Farrer Colony,L,289.0,Morana,15.8
3,Tea,686,1446,van Vogt City,L,176.0,Maunggu,17.1
4,Tea,1791,1230,Raymo's R & R & R,L,170.0,Cardea,17.3


In [83]:
listings_grouped = listings_ranked.sort_values(by=['distance', 'system', 'commodity', 'Sublight', 'station']) \
    .set_index(['system', 'commodity', 'station'])
listings_grouped

,,,supply,buy_price,Pad Size,Sublight,distance
system,commodity,station,,,,,
Azrael,Tea,Rafferty's Mobius,3009,1316,L,399.0,8.4
Mu Horologii,Tea,Seares Port,914,1232,L,9446.0,15.3
Morana,Tea,Farrer Colony,4447,1316,L,289.0,15.8
Maunggu,Tea,van Vogt City,686,1446,L,176.0,17.1
Cardea,Tea,Raymo's R & R & R,1791,1230,L,170.0,17.3
